# Цель ноутбука: создать неплохую ARTM

In [17]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))

import importlib # использую для перегрузки библиотек
import time
from IPython.display import clear_output
import random
import warnings

import pandas as pd
import json
import numpy as np
import artm

import data_preparing
import artm_model
import work_with_model as wwm

importlib.reload(data_preparing)
importlib.reload(artm_model)
importlib.reload(wwm)

warnings.filterwarnings("ignore")

clear_output(True)

artm.version() 0.9.2
artm.version() 0.9.2


In [5]:
inf = {}
inf['exp_name'] = 'weight_1_0_10_50'
inf['test_size'] = 0.1
inf['num_collection_passes'] = 15
inf['topic_number'] = 750
inf['data_path'] = "service_cards_tokenised_remont_only"

inf['docs'] = data_preparing.read_and_prepared(inf['data_path'])
inf['path_train'], inf['path_test'] = data_preparing.form_test_train_set(
                docs=inf['docs'], name=inf['exp_name'], test_size=inf['test_size'])
inf['path_castom'] = data_preparing.form_castom_set(inf['docs'], name=inf['exp_name'])
inf['model'], inf['theta_train'] = artm_model.create_and_learn_PLSA_class_ids_weigth(
                                            inf['path_train'], 
                                            topic_number=inf['topic_number'], 
                                            num_collection_passes=inf['num_collection_passes'], 
                                            weigths=[1., 0., 10., 50.]
                                         )

inf['theta_test'] = wwm.get_theta_from_vw(inf, inf['path_test'])
inf['theta_castom'] = wwm.get_theta_from_vw(inf, inf['path_castom'])
clear_output(True)

wwm.make_docs_dicts(inf)
wwm.form_card2topic(inf)
wwm.get_topic2rubric(inf)
#wwm.measure_accuracy_on_test(inf)
wwm.make_prediction_by_p_cd(inf, mode='castom')
inf['method_topic2rubric_card2rubric_castom'] = wwm.make_prediction_by_topic2rubric(inf, mode='castom')
wwm.make_prediction_by_p_cd(inf, mode='test')
inf['method_topic2rubric_card2rubric_test'] = wwm.make_prediction_by_topic2rubric(inf, mode='test')
wwm.accuracy_on_test(inf, inf['method_topic2rubric_card2rubric_test'])
wwm.accuracy_on_test(inf, inf['p_cd_card2rubric_test'])

50348  /  66104 0.76164831175118
44105  /  66104 0.6672062205010287


0.6672062205010287

In [8]:
scoreModeStick = []
scoreProbStick = []
tau_list = [1.0, 2.5e+2, 2.5e+3, 2.5e+4, 2.5e+5, 2.5e+6, 2.5e+7]
for tau in tau_list:
    inf_dec_1 = {}
    inf_dec_1['exp_name'] = 'weight_1_0_10_50'
    inf_dec_1['test_size'] = 0.1
    inf_dec_1['num_collection_passes'] = 15
    inf_dec_1['topic_number'] = 750
    inf_dec_1['data_path'] = "service_cards_tokenised_remont_only"

    inf_dec_1['docs'] = data_preparing.read_and_prepared(inf_dec_1['data_path'])
    inf_dec_1['path_train'], inf_dec_1['path_test'] = data_preparing.form_test_train_set(
                    docs=inf_dec_1['docs'], name=inf_dec_1['exp_name'], test_size=inf_dec_1['test_size'])
    inf_dec_1['path_castom'] = data_preparing.form_castom_set(inf_dec_1['docs'], name=inf_dec_1['exp_name'])
    inf_dec_1['model'], inf_dec_1['theta_train'] = artm_model.create_and_learn_ARTM_decorPhi_text(
                                                inf_dec_1['path_train'], 
                                                topic_number=inf_dec_1['topic_number'], 
                                                num_collection_passes=inf_dec_1['num_collection_passes'], 
                                                weigths=[1., 0., 10., 50.],
                                                decorTau=tau
                                             )

    inf_dec_1['theta_test'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_test'])
    inf_dec_1['theta_castom'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_castom'])
    clear_output(True)

    wwm.make_docs_dicts(inf_dec_1)
    wwm.form_card2topic(inf_dec_1)
    wwm.get_topic2rubric(inf_dec_1)
    #wwm.measure_accuracy_on_test(inf_dec_1)
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='castom')
    inf_dec_1['method_topic2rubric_card2rubric_castom'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='castom')
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='test')
    inf_dec_1['method_topic2rubric_card2rubric_test'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='test')
    scoreModeStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test']))
    scoreProbStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['p_cd_card2rubric_test']))
print(tau_list)
print(scoreModeStick)
print(scoreProbStick)

4798  /  66104 0.07258259711969019
6716  /  66104 0.10159748275444754
[1.0, 250.0, 2500.0, 25000.0, 250000.0, 2500000.0, 25000000.0]
[0.7616180563959821, 0.6914256323369237, 0.6665859857194724, 0.09864758562265522, 0.06170579692605591, 0.06064685949413046, 0.07258259711969019]
[0.6672062205010287, 0.6254840856831659, 0.6290693452741135, 0.11840433256686433, 0.09808786155149461, 0.10025111944814231, 0.10159748275444754]


In [3]:
scoreModeStick = []
scoreProbStick = []
tau_list = [1.0, 2.5e+2, 2.5e+3, 2.5e+4, 2.5e+5, 2.5e+6, 2.5e+7]
for tau in tau_list:
    inf_dec_1 = {}
    inf_dec_1['exp_name'] = 'weight_1_0_10_50'
    inf_dec_1['test_size'] = 0.1
    inf_dec_1['num_collection_passes'] = 15
    inf_dec_1['topic_number'] = 750
    inf_dec_1['data_path'] = "service_cards_tokenised_remont_only"

    inf_dec_1['docs'] = data_preparing.read_and_prepared(inf_dec_1['data_path'])
    inf_dec_1['path_train'], inf_dec_1['path_test'] = data_preparing.form_test_train_set(
                    docs=inf_dec_1['docs'], name=inf_dec_1['exp_name'], test_size=inf_dec_1['test_size'])
    inf_dec_1['path_castom'] = data_preparing.form_castom_set(inf_dec_1['docs'], name=inf_dec_1['exp_name'])
    inf_dec_1['model'], inf_dec_1['theta_train'] = artm_model.create_and_learn_ARTM_decorPhi_modal(
                                                inf_dec_1['path_train'], 
                                                topic_number=inf_dec_1['topic_number'], 
                                                num_collection_passes=inf_dec_1['num_collection_passes'], 
                                                weigths=[1., 0., 10., 50.],
                                                decorTau=tau
                                             )

    inf_dec_1['theta_test'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_test'])
    inf_dec_1['theta_castom'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_castom'])
    clear_output(True)

    wwm.make_docs_dicts(inf_dec_1)
    wwm.form_card2topic(inf_dec_1)
    wwm.get_topic2rubric(inf_dec_1)
    #wwm.measure_accuracy_on_test(inf_dec_1)
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='castom')
    inf_dec_1['method_topic2rubric_card2rubric_castom'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='castom')
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='test')
    inf_dec_1['method_topic2rubric_card2rubric_test'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='test')
    scoreModeStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test']))
    scoreProbStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['p_cd_card2rubric_test']))
print(tau_list)
print(scoreModeStick)
print(scoreProbStick)

41220  /  66104 0.6235628706281012
33  /  66104 0.0004992133607648554
[1.0, 250.0, 2500.0, 25000.0, 250000.0, 2500000.0, 25000000.0]
[0.7627223768607043, 0.6677961999273871, 0.7116210819315019, 0.6203104199443301, 0.613139900762435, 0.6238805518576788, 0.6235628706281012]
[0.6671457097906329, 0.6950714026382669, 0.7356135786034128, 0.05055669853564081, 0.0004992133607648554, 0.0004992133607648554, 0.0004992133607648554]


In [4]:
scoreModeStick = []
scoreProbStick = []
tau_list = [0.5, 50, 500, 5000, 10000, 50000]
for tau in tau_list:
    inf_dec_1 = {}
    inf_dec_1['exp_name'] = 'weight_1_0_10_50'
    inf_dec_1['test_size'] = 0.1
    inf_dec_1['num_collection_passes'] = 15
    inf_dec_1['topic_number'] = 750
    inf_dec_1['data_path'] = "service_cards_tokenised_remont_only"

    inf_dec_1['docs'] = data_preparing.read_and_prepared(inf_dec_1['data_path'])
    inf_dec_1['path_train'], inf_dec_1['path_test'] = data_preparing.form_test_train_set(
                    docs=inf_dec_1['docs'], name=inf_dec_1['exp_name'], test_size=inf_dec_1['test_size'])
    inf_dec_1['path_castom'] = data_preparing.form_castom_set(inf_dec_1['docs'], name=inf_dec_1['exp_name'])
    inf_dec_1['model'], inf_dec_1['theta_train'] = artm_model.create_and_learn_ARTM_decorPhi_modal(
                                                inf_dec_1['path_train'], 
                                                topic_number=inf_dec_1['topic_number'], 
                                                num_collection_passes=inf_dec_1['num_collection_passes'], 
                                                weigths=[1., 0., 10., 50.],
                                                decorTau=tau
                                             )

    inf_dec_1['theta_test'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_test'])
    inf_dec_1['theta_castom'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_castom'])
    clear_output(True)

    wwm.make_docs_dicts(inf_dec_1)
    wwm.form_card2topic(inf_dec_1)
    wwm.get_topic2rubric(inf_dec_1)
    #wwm.measure_accuracy_on_test(inf_dec_1)
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='castom')
    inf_dec_1['method_topic2rubric_card2rubric_castom'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='castom')
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='test')
    inf_dec_1['method_topic2rubric_card2rubric_test'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='test')
    scoreModeStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test']))
    scoreProbStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['p_cd_card2rubric_test']))
print(tau_list)
print(scoreModeStick)
print(scoreProbStick)

41239  /  66104 0.6238502965024809
1707  /  66104 0.025822945661382064
[0.5, 50, 500, 5000, 10000, 50000]
[0.7612398644560087, 0.7563687522691517, 0.6940881035943361, 0.716946024446327, 0.718806728790996, 0.6238502965024809]
[0.6671457097906329, 0.6547561418371052, 0.727187462180806, 0.7547500907660656, 0.7387752632215903, 0.025822945661382064]


In [5]:
scoreModeStick = []
scoreProbStick = []
tau_list = [3000, 4000, 6000]
for tau in tau_list:
    inf_dec_1 = {}
    inf_dec_1['exp_name'] = 'weight_1_0_10_50'
    inf_dec_1['test_size'] = 0.1
    inf_dec_1['num_collection_passes'] = 15
    inf_dec_1['topic_number'] = 750
    inf_dec_1['data_path'] = "service_cards_tokenised_remont_only"

    inf_dec_1['docs'] = data_preparing.read_and_prepared(inf_dec_1['data_path'])
    inf_dec_1['path_train'], inf_dec_1['path_test'] = data_preparing.form_test_train_set(
                    docs=inf_dec_1['docs'], name=inf_dec_1['exp_name'], test_size=inf_dec_1['test_size'])
    inf_dec_1['path_castom'] = data_preparing.form_castom_set(inf_dec_1['docs'], name=inf_dec_1['exp_name'])
    inf_dec_1['model'], inf_dec_1['theta_train'] = artm_model.create_and_learn_ARTM_decorPhi_modal(
                                                inf_dec_1['path_train'], 
                                                topic_number=inf_dec_1['topic_number'], 
                                                num_collection_passes=inf_dec_1['num_collection_passes'], 
                                                weigths=[1., 0., 10., 50.],
                                                decorTau=tau
                                             )

    inf_dec_1['theta_test'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_test'])
    inf_dec_1['theta_castom'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_castom'])
    clear_output(True)

    wwm.make_docs_dicts(inf_dec_1)
    wwm.form_card2topic(inf_dec_1)
    wwm.get_topic2rubric(inf_dec_1)
    #wwm.measure_accuracy_on_test(inf_dec_1)
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='castom')
    inf_dec_1['method_topic2rubric_card2rubric_castom'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='castom')
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='test')
    inf_dec_1['method_topic2rubric_card2rubric_test'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='test')
    scoreModeStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test']))
    scoreProbStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['p_cd_card2rubric_test']))
print(tau_list)
print(scoreModeStick)
print(scoreProbStick)

48350  /  66104 0.7314232119085078
50579  /  66104 0.765142805276534
[3000, 4000, 6000]
[0.6136996248335955, 0.7140263826697325, 0.7314232119085078]
[0.6306123683892049, 0.7443573762555973, 0.765142805276534]


# при decorTau 8000 проб метод - хорош

In [3]:
scoreModeStick = []
scoreProbStick = []
tau_list = [7000, 8000, 9000]
for tau in tau_list:
    inf_dec_1 = {}
    inf_dec_1['exp_name'] = 'weight_1_0_10_50'
    inf_dec_1['test_size'] = 0.1
    inf_dec_1['num_collection_passes'] = 15
    inf_dec_1['topic_number'] = 750
    inf_dec_1['data_path'] = "service_cards_tokenised_remont_only"

    inf_dec_1['docs'] = data_preparing.read_and_prepared(inf_dec_1['data_path'])
    inf_dec_1['path_train'], inf_dec_1['path_test'] = data_preparing.form_test_train_set(
                    docs=inf_dec_1['docs'], name=inf_dec_1['exp_name'], test_size=inf_dec_1['test_size'])
    inf_dec_1['path_castom'] = data_preparing.form_castom_set(inf_dec_1['docs'], name=inf_dec_1['exp_name'])
    inf_dec_1['model'], inf_dec_1['theta_train'] = artm_model.create_and_learn_ARTM_decorPhi_modal(
                                                inf_dec_1['path_train'], 
                                                topic_number=inf_dec_1['topic_number'], 
                                                num_collection_passes=inf_dec_1['num_collection_passes'], 
                                                weigths=[1., 0., 10., 50.],
                                                decorTau=tau
                                             )

    inf_dec_1['theta_test'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_test'])
    inf_dec_1['theta_castom'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_castom'])
    clear_output(True)

    wwm.make_docs_dicts(inf_dec_1)
    wwm.form_card2topic(inf_dec_1)
    wwm.get_topic2rubric(inf_dec_1)
    #wwm.measure_accuracy_on_test(inf_dec_1)
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='castom')
    inf_dec_1['method_topic2rubric_card2rubric_castom'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='castom')
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='test')
    inf_dec_1['method_topic2rubric_card2rubric_test'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='test')
    scoreModeStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test']))
    scoreProbStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['p_cd_card2rubric_test']))
print(tau_list)
print(scoreModeStick)
print(scoreProbStick)

48867  /  66104 0.7392442212271572
50106  /  66104 0.7579874137722377
[7000, 8000, 9000]
[0.7194572189277502, 0.7207430715236597, 0.7392442212271572]
[0.7623593125983299, 0.7637964419702288, 0.7579874137722377]


In [10]:
scoreModeStick = []
scoreProbStick = []
tau_list = [1]
for tau in tau_list:
    inf_dec_1 = {}
    inf_dec_1['exp_name'] = 'weight_backround_topics'
    inf_dec_1['test_size'] = 0.1
    inf_dec_1['num_collection_passes'] = 15
    inf_dec_1['topic_number'] = 750
    inf_dec_1['data_path'] = "service_cards_tokenised_remont_only"

    inf_dec_1['docs'] = data_preparing.read_and_prepared(inf_dec_1['data_path'])
    inf_dec_1['path_train'], inf_dec_1['path_test'] = data_preparing.form_test_train_set(
                    docs=inf_dec_1['docs'], name=inf_dec_1['exp_name'], test_size=inf_dec_1['test_size'])
    inf_dec_1['path_castom'] = data_preparing.form_castom_set(inf_dec_1['docs'], name=inf_dec_1['exp_name'])
    inf_dec_1['model'], inf_dec_1['theta_train'] = artm_model.create_and_learn__ARTM_background_topics(
                                                inf_dec_1['path_train'], 
                                                topic_number=inf_dec_1['topic_number'], 
                                                num_collection_passes=inf_dec_1['num_collection_passes'], 
                                                weigths=[1., 0., 10., 50.],
                                                decorTau=tau
                                             )

    inf_dec_1['theta_test'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_test'])
    inf_dec_1['theta_castom'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_castom'])
    clear_output(True)

    wwm.make_docs_dicts(inf_dec_1)
    wwm.form_card2topic(inf_dec_1)
    wwm.get_topic2rubric(inf_dec_1)
    #wwm.measure_accuracy_on_test(inf_dec_1)
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='castom')
    inf_dec_1['method_topic2rubric_card2rubric_castom'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='castom')
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='test')
    inf_dec_1['method_topic2rubric_card2rubric_test'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='test')
    scoreModeStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test']))
    scoreProbStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['p_cd_card2rubric_test']))
print(tau_list)
print(scoreModeStick)
print(scoreProbStick)

Exception ignored in: <function ARTM.__del__ at 0x7fbfe56665f0>
Traceback (most recent call last):
  File "/home/kozlinskii/python-virtual-environments/bigartm_9/lib/python3.7/site-packages/artm/artm_model.py", line 293, in __del__
    self.dispose()
  File "/home/kozlinskii/python-virtual-environments/bigartm_9/lib/python3.7/site-packages/artm/artm_model.py", line 285, in dispose
    if self._master is not None:
AttributeError: 'ARTM' object has no attribute '_master'


10689  /  66104 0.16169974585501634
11535  /  66104 0.17449776110371537
[1]
[0.16169974585501634]
[0.17449776110371537]


In [12]:
scoreModeStick = []
scoreProbStick = []
tau_list = [1]
for tau in tau_list:
    inf_dec_1 = {}
    inf_dec_1['exp_name'] = 'weight_backround_topics'
    inf_dec_1['test_size'] = 0.1
    inf_dec_1['num_collection_passes'] = 15
    inf_dec_1['topic_number'] = 750
    inf_dec_1['data_path'] = "service_cards_tokenised_remont_only"

    inf_dec_1['docs'] = data_preparing.read_and_prepared(inf_dec_1['data_path'])
    inf_dec_1['path_train'], inf_dec_1['path_test'] = data_preparing.form_test_train_set(
                    docs=inf_dec_1['docs'], name=inf_dec_1['exp_name'], test_size=inf_dec_1['test_size'])
    inf_dec_1['path_castom'] = data_preparing.form_castom_set(inf_dec_1['docs'], name=inf_dec_1['exp_name'])
    inf_dec_1['model'], inf_dec_1['theta_train'] = artm_model.create_and_learn__ARTM_background_topics(
                                                inf_dec_1['path_train'], 
                                                topic_number=inf_dec_1['topic_number'], 
                                                num_collection_passes=inf_dec_1['num_collection_passes'], 
                                                weigths=[1., 0., 10., 50.],
                                                decorTau=tau,
                                                back_ThetaReg=0,
                                                back_PhiReg = 0,
                                             )

    inf_dec_1['theta_test'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_test'])
    inf_dec_1['theta_castom'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_castom'])
    #clear_output(True)

    wwm.make_docs_dicts(inf_dec_1)
    wwm.form_card2topic(inf_dec_1)
    wwm.get_topic2rubric(inf_dec_1)
    #wwm.measure_accuracy_on_test(inf_dec_1)
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='castom')
    inf_dec_1['method_topic2rubric_card2rubric_castom'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='castom')
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='test')
    inf_dec_1['method_topic2rubric_card2rubric_test'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='test')
    scoreModeStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test']))
    scoreProbStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['p_cd_card2rubric_test']))
    wwm.calc_all_scores(inf, inf['method_topic2rubric_card2rubric_test'])
print(tau_list)
print(scoreModeStick)
print(scoreProbStick)

50419  /  66104 0.7627223768607043
44100  /  66104 0.667130582113034
[1]
[0.7627223768607043]
[0.667130582113034]


In [15]:
scoreModeStick = []
scoreProbStick = []
tau_list = [0.001, 0.01, 0.1, 1 ]
for tau in tau_list:
    inf_dec_1 = {}
    inf_dec_1['exp_name'] = 'weight_backround_topics'
    inf_dec_1['test_size'] = 0.1
    inf_dec_1['num_collection_passes'] = 15
    inf_dec_1['topic_number'] = 750
    inf_dec_1['data_path'] = "service_cards_tokenised_remont_only"

    inf_dec_1['docs'] = data_preparing.read_and_prepared(inf_dec_1['data_path'])
    inf_dec_1['path_train'], inf_dec_1['path_test'] = data_preparing.form_test_train_set(
                    docs=inf_dec_1['docs'], name=inf_dec_1['exp_name'], test_size=inf_dec_1['test_size'])
    inf_dec_1['path_castom'] = data_preparing.form_castom_set(inf_dec_1['docs'], name=inf_dec_1['exp_name'])
    inf_dec_1['model'], inf_dec_1['theta_train'] = artm_model.create_and_learn__ARTM_background_topics(
                                                inf_dec_1['path_train'], 
                                                topic_number=inf_dec_1['topic_number'], 
                                                num_collection_passes=inf_dec_1['num_collection_passes'], 
                                                weigths=[1., 0., 10., 50.],
                                                decorTau=1,
                                                back_ThetaReg=0,
                                                back_PhiReg = tau,
                                             )

    inf_dec_1['theta_test'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_test'])
    inf_dec_1['theta_castom'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_castom'])
    #clear_output(True)

    wwm.make_docs_dicts(inf_dec_1)
    wwm.form_card2topic(inf_dec_1)
    wwm.get_topic2rubric(inf_dec_1)
    #wwm.measure_accuracy_on_test(inf_dec_1)
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='castom')
    inf_dec_1['method_topic2rubric_card2rubric_castom'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='castom')
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='test')
    inf_dec_1['method_topic2rubric_card2rubric_test'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='test')
    scoreModeStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test']))
    scoreProbStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['p_cd_card2rubric_test']))
    wwm.calc_all_scores(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test'])
print(tau_list)
print(scoreModeStick)
print(scoreProbStick)

50414  /  66104 0.7626467384727097
44101  /  66104 0.6671457097906329
weight_backround_topics
precision 0.7652979127134725
recall 0.7626467384727097
tfind, ffind, ffindr, fignore, tignore, n := 50414 15461 0 229 0 66104
50384  /  66104 0.7621929081447416
44101  /  66104 0.6671457097906329
weight_backround_topics
precision 0.7647380244065327
recall 0.7621929081447416
tfind, ffind, ffindr, fignore, tignore, n := 50384 15500 0 220 0 66104
50402  /  66104 0.7624652063415225
44106  /  66104 0.6672213481786277
weight_backround_topics
precision 0.7650112318620605
recall 0.7624652063415225
tfind, ffind, ffindr, fignore, tignore, n := 50402 15482 0 220 0 66104
50331  /  66104 0.761391141231998
44033  /  66104 0.6661170277139054
weight_backround_topics
precision 0.7639335802319228
recall 0.761391141231998
tfind, ffind, ffindr, fignore, tignore, n := 50331 15553 0 220 0 66104
[0.001, 0.01, 0.1, 1]
[0.7626467384727097, 0.7621929081447416, 0.7624652063415225, 0.761391141231998]
[0.6671457097906329,

In [18]:
scoreModeStick = []
scoreProbStick = []
tau_list = [2, -1, 100]
for tau in tau_list:
    inf_dec_1 = {}
    inf_dec_1['exp_name'] = 'weight_backround_topics'
    inf_dec_1['test_size'] = 0.1
    inf_dec_1['num_collection_passes'] = 15
    inf_dec_1['topic_number'] = 750
    inf_dec_1['data_path'] = "service_cards_tokenised_remont_only"

    inf_dec_1['docs'] = data_preparing.read_and_prepared(inf_dec_1['data_path'])
    inf_dec_1['path_train'], inf_dec_1['path_test'] = data_preparing.form_test_train_set(
                    docs=inf_dec_1['docs'], name=inf_dec_1['exp_name'], test_size=inf_dec_1['test_size'])
    inf_dec_1['path_castom'] = data_preparing.form_castom_set(inf_dec_1['docs'], name=inf_dec_1['exp_name'])
    inf_dec_1['model'], inf_dec_1['theta_train'] = artm_model.create_and_learn__ARTM_background_topics(
                                                inf_dec_1['path_train'], 
                                                topic_number=inf_dec_1['topic_number'], 
                                                num_collection_passes=inf_dec_1['num_collection_passes'], 
                                                weigths=[1., 0., 10., 50.],
                                                decorTau=1,
                                                back_ThetaReg=0,
                                                back_PhiReg = tau,
                                                obj_PhiReg = 0,
                                                obj_ThetaReg = 0,
                                             )

    inf_dec_1['theta_test'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_test'])
    inf_dec_1['theta_castom'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_castom'])
    #clear_output(True)

    wwm.make_docs_dicts(inf_dec_1)
    wwm.form_card2topic(inf_dec_1)
    wwm.get_topic2rubric(inf_dec_1)
    #wwm.measure_accuracy_on_test(inf_dec_1)
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='castom')
    inf_dec_1['method_topic2rubric_card2rubric_castom'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='castom')
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='test')
    inf_dec_1['method_topic2rubric_card2rubric_test'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='test')
    scoreModeStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test']))
    scoreProbStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['p_cd_card2rubric_test']))
    wwm.calc_all_scores(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test'])
print(tau_list)
print(scoreModeStick)
print(scoreProbStick)

50328  /  66104 0.7613457581992013
44033  /  66104 0.6661170277139054
weight_backround_topics
precision 0.763876451392578
recall 0.7613342409802587
tfind, ffind, ffindr, fignore, tignore, n := 50328 15556 0 220 0 66104
50405  /  66104 0.7625105893743193
43860  /  66104 0.6634999394892896
weight_backround_topics
precision 0.7651729058505632
recall 0.7624990545344528
tfind, ffind, ffindr, fignore, tignore, n := 50405 15468 0 231 0 66104
50326  /  66104 0.7613155028440034
44033  /  66104 0.6661170277139054
weight_backround_topics
precision 0.7638460954693784
recall 0.7613039860827472
tfind, ffind, ffindr, fignore, tignore, n := 50326 15558 0 220 0 66104
[2, -1, 100]
[0.7613457581992013, 0.7625105893743193, 0.7613155028440034]
[0.6661170277139054, 0.6634999394892896, 0.6661170277139054]


In [21]:
scoreModeStick = []
scoreProbStick = []
tau_list = [1000, 10000, 100000]
for tau in tau_list:
    inf_dec_1 = {}
    inf_dec_1['exp_name'] = 'weight_backround_topics'
    inf_dec_1['test_size'] = 0.1
    inf_dec_1['num_collection_passes'] = 15
    inf_dec_1['topic_number'] = 750
    inf_dec_1['data_path'] = "service_cards_tokenised_remont_only"

    inf_dec_1['docs'] = data_preparing.read_and_prepared(inf_dec_1['data_path'])
    inf_dec_1['path_train'], inf_dec_1['path_test'] = data_preparing.form_test_train_set(
                    docs=inf_dec_1['docs'], name=inf_dec_1['exp_name'], test_size=inf_dec_1['test_size'])
    inf_dec_1['path_castom'] = data_preparing.form_castom_set(inf_dec_1['docs'], name=inf_dec_1['exp_name'])
    inf_dec_1['model'], inf_dec_1['theta_train'] = artm_model.create_and_learn__ARTM_background_topics(
                                                inf_dec_1['path_train'], 
                                                topic_number=inf_dec_1['topic_number'], 
                                                num_collection_passes=inf_dec_1['num_collection_passes'], 
                                                weigths=[1., 0., 10., 50.],
                                                decorTau=1,
                                                back_ThetaReg=0,
                                                back_PhiReg = tau,
                                                obj_PhiReg = 0,
                                                obj_ThetaReg = 0,
                                             )

    inf_dec_1['theta_test'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_test'])
    inf_dec_1['theta_castom'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_castom'])
    #clear_output(True)

    wwm.make_docs_dicts(inf_dec_1)
    wwm.form_card2topic(inf_dec_1)
    wwm.get_topic2rubric(inf_dec_1)
    #wwm.measure_accuracy_on_test(inf_dec_1)
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='castom')
    inf_dec_1['method_topic2rubric_card2rubric_castom'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='castom')
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='test')
    inf_dec_1['method_topic2rubric_card2rubric_test'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='test')
    scoreModeStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test']))
    scoreProbStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['p_cd_card2rubric_test']))
    wwm.calc_all_scores(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test'])
print(tau_list)
print(scoreModeStick)
print(scoreProbStick)

50326  /  66104 0.7613155028440034
44033  /  66104 0.6661170277139054
weight_backround_topics
precision 0.7638460954693784
recall 0.7613039860827472
tfind, ffind, ffindr, fignore, tignore, n := 50326 15558 0 220 0 66104
50326  /  66104 0.7613155028440034
44033  /  66104 0.6661170277139054
weight_backround_topics
precision 0.7638460954693784
recall 0.7613039860827472
tfind, ffind, ffindr, fignore, tignore, n := 50326 15558 0 220 0 66104
50326  /  66104 0.7613155028440034
44033  /  66104 0.6661170277139054
weight_backround_topics
precision 0.7638460954693784
recall 0.7613039860827472
tfind, ffind, ffindr, fignore, tignore, n := 50326 15558 0 220 0 66104
[1000, 10000, 100000]
[0.7613155028440034, 0.7613155028440034, 0.7613155028440034]
[0.6661170277139054, 0.6661170277139054, 0.6661170277139054]


In [16]:
scoreModeStick = []
scoreProbStick = []
tau_list = [0.001, 0.01, 0.1, 1 ]
for tau in tau_list:
    inf_dec_1 = {}
    inf_dec_1['exp_name'] = 'weight_backround_topics'
    inf_dec_1['test_size'] = 0.1
    inf_dec_1['num_collection_passes'] = 15
    inf_dec_1['topic_number'] = 750
    inf_dec_1['data_path'] = "service_cards_tokenised_remont_only"

    inf_dec_1['docs'] = data_preparing.read_and_prepared(inf_dec_1['data_path'])
    inf_dec_1['path_train'], inf_dec_1['path_test'] = data_preparing.form_test_train_set(
                    docs=inf_dec_1['docs'], name=inf_dec_1['exp_name'], test_size=inf_dec_1['test_size'])
    inf_dec_1['path_castom'] = data_preparing.form_castom_set(inf_dec_1['docs'], name=inf_dec_1['exp_name'])
    inf_dec_1['model'], inf_dec_1['theta_train'] = artm_model.create_and_learn__ARTM_background_topics(
                                                inf_dec_1['path_train'], 
                                                topic_number=inf_dec_1['topic_number'], 
                                                num_collection_passes=inf_dec_1['num_collection_passes'], 
                                                weigths=[1., 0., 10., 50.],
                                                decorTau=1,
                                                back_ThetaReg= tau,
                                                back_PhiReg = 0,
                                             )

    inf_dec_1['theta_test'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_test'])
    inf_dec_1['theta_castom'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_castom'])
    #clear_output(True)

    wwm.make_docs_dicts(inf_dec_1)
    wwm.form_card2topic(inf_dec_1)
    wwm.get_topic2rubric(inf_dec_1)
    #wwm.measure_accuracy_on_test(inf_dec_1)
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='castom')
    inf_dec_1['method_topic2rubric_card2rubric_castom'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='castom')
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='test')
    inf_dec_1['method_topic2rubric_card2rubric_test'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='test')
    scoreModeStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test']))
    scoreProbStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['p_cd_card2rubric_test']))
    wwm.calc_all_scores(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test'])
print(tau_list)
print(scoreModeStick)
print(scoreProbStick)

50416  /  66104 0.7626769938279075
44101  /  66104 0.6671457097906329
weight_backround_topics
precision 0.7653166555346408
recall 0.7626769938279075
tfind, ffind, ffindr, fignore, tignore, n := 50416 15460 0 228 0 66104
50529  /  66104 0.7643864213965872
43966  /  66104 0.6651034733147767
weight_backround_topics
precision 0.7669388622427297
recall 0.7643864213965872
tfind, ffind, ffindr, fignore, tignore, n := 50529 15355 0 220 0 66104
49922  /  66104 0.7552039210940337
44177  /  66104 0.668295413288152
weight_backround_topics
precision 0.7608205315776639
recall 0.7552039210940337
tfind, ffind, ffindr, fignore, tignore, n := 49922 15694 0 488 0 66104
34820  /  66104 0.5267457339949171
36661  /  66104 0.5545957884545565
weight_backround_topics
precision 0.5274558812391124
recall 0.5267457339949171
tfind, ffind, ffindr, fignore, tignore, n := 34820 31195 0 89 0 66104
[0.001, 0.01, 0.1, 1]
[0.7626769938279075, 0.7643864213965872, 0.7552039210940337, 0.5267457339949171]
[0.6671457097906329

In [19]:
scoreModeStick = []
scoreProbStick = []
tau_list = [-0.001, -0.01, -0.1, -1, -10 ]
for tau in tau_list:
    inf_dec_1 = {}
    inf_dec_1['exp_name'] = 'weight_backround_topics'
    inf_dec_1['test_size'] = 0.1
    inf_dec_1['num_collection_passes'] = 15
    inf_dec_1['topic_number'] = 750
    inf_dec_1['data_path'] = "service_cards_tokenised_remont_only"

    inf_dec_1['docs'] = data_preparing.read_and_prepared(inf_dec_1['data_path'])
    inf_dec_1['path_train'], inf_dec_1['path_test'] = data_preparing.form_test_train_set(
                    docs=inf_dec_1['docs'], name=inf_dec_1['exp_name'], test_size=inf_dec_1['test_size'])
    inf_dec_1['path_castom'] = data_preparing.form_castom_set(inf_dec_1['docs'], name=inf_dec_1['exp_name'])
    inf_dec_1['model'], inf_dec_1['theta_train'] = artm_model.create_and_learn__ARTM_background_topics(
                                                inf_dec_1['path_train'], 
                                                topic_number=inf_dec_1['topic_number'], 
                                                num_collection_passes=inf_dec_1['num_collection_passes'], 
                                                weigths=[1., 0., 10., 50.],
                                                decorTau=1,
                                                back_ThetaReg= 0,
                                                back_PhiReg = 0,
                                                obj_PhiReg = tau,
                                                obj_ThetaReg = 0,
                                             )

    inf_dec_1['theta_test'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_test'])
    inf_dec_1['theta_castom'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_castom'])
    #clear_output(True)

    wwm.make_docs_dicts(inf_dec_1)
    wwm.form_card2topic(inf_dec_1)
    wwm.get_topic2rubric(inf_dec_1)
    #wwm.measure_accuracy_on_test(inf_dec_1)
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='castom')
    inf_dec_1['method_topic2rubric_card2rubric_castom'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='castom')
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='test')
    inf_dec_1['method_topic2rubric_card2rubric_test'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='test')
    scoreModeStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test']))
    scoreProbStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['p_cd_card2rubric_test']))
    wwm.calc_all_scores(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test'])
print(tau_list)
print(scoreModeStick)
print(scoreProbStick)

50498  /  66104 0.7639174633910202
44004  /  66104 0.6656783250635362
weight_backround_topics
precision 0.7697396500213401
recall 0.7639059072687391
tfind, ffind, ffindr, fignore, tignore, n := 50498 15105 0 501 0 66104
49668  /  66104 0.7513614909839041
43711  /  66104 0.6612459155270483
weight_backround_topics
precision 0.7530589037980441
recall 0.7513501248014522
tfind, ffind, ffindr, fignore, tignore, n := 49668 16286 0 150 0 66104
47286  /  66104 0.715327362943241
43374  /  66104 0.6561478881762072
weight_backround_topics
precision 0.7193099881347166
recall 0.7153165418652144
tfind, ffind, ffindr, fignore, tignore, n := 47286 18451 0 367 0 66104
43070  /  66104 0.651549074186131
40983  /  66104 0.6199776110371535
weight_backround_topics
precision 0.6544100888855124
recall 0.6515392179108993
tfind, ffind, ffindr, fignore, tignore, n := 43070 22744 0 290 0 66104
18359  /  66104 0.2777290330388479
21059  /  66104 0.3185737625559724
weight_backround_topics
precision 0.2783269154968011

In [20]:
scoreModeStick = []
scoreProbStick = []
tau_list = [-0.001, -0.01, -0.1, -1, -10 ]
for tau in tau_list:
    inf_dec_1 = {}
    inf_dec_1['exp_name'] = 'weight_backround_topics'
    inf_dec_1['test_size'] = 0.1
    inf_dec_1['num_collection_passes'] = 15
    inf_dec_1['topic_number'] = 750
    inf_dec_1['data_path'] = "service_cards_tokenised_remont_only"

    inf_dec_1['docs'] = data_preparing.read_and_prepared(inf_dec_1['data_path'])
    inf_dec_1['path_train'], inf_dec_1['path_test'] = data_preparing.form_test_train_set(
                    docs=inf_dec_1['docs'], name=inf_dec_1['exp_name'], test_size=inf_dec_1['test_size'])
    inf_dec_1['path_castom'] = data_preparing.form_castom_set(inf_dec_1['docs'], name=inf_dec_1['exp_name'])
    inf_dec_1['model'], inf_dec_1['theta_train'] = artm_model.create_and_learn__ARTM_background_topics(
                                                inf_dec_1['path_train'], 
                                                topic_number=inf_dec_1['topic_number'], 
                                                num_collection_passes=inf_dec_1['num_collection_passes'], 
                                                weigths=[1., 0., 10., 50.],
                                                decorTau=1,
                                                back_ThetaReg= 0,
                                                back_PhiReg = 0,
                                                obj_PhiReg = 0,
                                                obj_ThetaReg = tau,
                                             )

    inf_dec_1['theta_test'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_test'])
    inf_dec_1['theta_castom'] = wwm.get_theta_from_vw(inf_dec_1, inf_dec_1['path_castom'])
    #clear_output(True)

    wwm.make_docs_dicts(inf_dec_1)
    wwm.form_card2topic(inf_dec_1)
    wwm.get_topic2rubric(inf_dec_1)
    #wwm.measure_accuracy_on_test(inf_dec_1)
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='castom')
    inf_dec_1['method_topic2rubric_card2rubric_castom'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='castom')
    wwm.make_prediction_by_p_cd(inf_dec_1, mode='test')
    inf_dec_1['method_topic2rubric_card2rubric_test'] = wwm.make_prediction_by_topic2rubric(inf_dec_1, mode='test')
    scoreModeStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test']))
    scoreProbStick.append(wwm.accuracy_on_test(inf_dec_1, inf_dec_1['p_cd_card2rubric_test']))
    wwm.calc_all_scores(inf_dec_1, inf_dec_1['method_topic2rubric_card2rubric_test'])
print(tau_list)
print(scoreModeStick)
print(scoreProbStick)

50487  /  66104 0.7637510589374319
44107  /  66104 0.6672364758562266
weight_backround_topics
precision 0.7663944380350962
recall 0.7637395053324257
tfind, ffind, ffindr, fignore, tignore, n := 50487 15388 0 229 0 66104
50581  /  66104 0.7651730606317318
44643  /  66104 0.6753449110492558
weight_backround_topics
precision 0.7689419276375798
recall 0.7651614855154678
tfind, ffind, ffindr, fignore, tignore, n := 50581 15198 0 325 0 66104
48047  /  66104 0.7268395255960305
44772  /  66104 0.6772963814595183
weight_backround_topics
precision 0.7278966185916859
recall 0.7268285303683534
tfind, ffind, ffindr, fignore, tignore, n := 48047 17960 0 97 0 66104
1009  /  66104 0.015263826697325426
2396  /  66104 0.03624591552704829
weight_backround_topics
precision 0.015265443212248665
recall 0.015263595794569245
tfind, ffind, ffindr, fignore, tignore, n := 1009 65087 0 8 0 66104
813  /  66104 0.012298801887934164
488  /  66104 0.007382306668280285
weight_backround_topics
precision 0.0122986158384

#### Почему же за 76% ни разу не перевалило?

In [ ]:
посмотреть оставшиеся проценты и понять все ли впорядке с данными